In [2]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Mistral 7B Base

In [37]:
model_name = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

Device set to use cuda:0


In [39]:
first_prompt = """\
You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and say if it is a meeting scheduling request or not.

Do not provide any explanations and only respond with one word: 'yes' or 'no'.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Answer: yes
Text: The company generated net sales of 11.3 million euro this year.
Answer: no

Your TEXT to analyse:
TEXT: {text}
Answer: """


In [40]:
text = "Schedule a meeting with Marc tomorrow morning or Thursday afternoon"
prompt = first_prompt.format(text=text)
print(prompt)

You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and say if it is a meeting scheduling request or not.

Do not provide any explanations and only respond with one word: 'yes' or 'no'.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Answer: yes
Text: The company generated net sales of 11.3 million euro this year.
Answer: no

Your TEXT to analyse:
TEXT: Schedule a meeting with Marc tomorrow morning or Thursday afternoon
Answer: 


In [41]:
response = pipe(
    prompt,
    do_sample=False,
    max_new_tokens=2, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and say if it is a meeting scheduling request or not.

Do not provide any explanations and only respond with one word: 'yes' or 'no'.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Answer: yes
Text: The company generated net sales of 11.3 million euro this year.
Answer: no

Your TEXT to analyse:
TEXT: Schedule a meeting with Marc tomorrow morning or Thursday afternoon
Answer:  yes



In [42]:
if "yes" in response[0]["generated_text"][-5:]:
    predicted_label = "meeting scheduling request"
elif "no" in response[0]["generated_text"][-5:]:
    predicted_label = "other"
else:
    predicted_label = "FAIL"

predicted_label

'meeting scheduling request'

In [43]:
second_prompt = """\
You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and identify with whom the meeting should be organized and which slots are requested by the user.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Participants: Florian
Slots: anyday this week or next week, the afternoon

Text: Organize a meeting with John next Monday at 2 PM
Participants: John
Slots: Monday, 2P M

Your TEXT to analyse:
TEXT: {text}
Participants: """


In [44]:
text = "Schedule a meeting with Marc tomorrow morning or Thursday afternoon"
prompt = second_prompt.format(text=text)
print(prompt)

You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and identify with whom the meeting should be organized and which slots are requested by the user.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Participants: Florian
Slots: anyday this week or next week, the afternoon

Text: Organize a meeting with John next Monday at 2 PM
Participants: John
Slots: Monday, 2P M

Your TEXT to analyse:
TEXT: Schedule a meeting with Marc tomorrow morning or Thursday afternoon
Participants: 


In [45]:
response = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=128, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a meeting assistant who schedules meetings based on the user's request. Meetings must be scheduled on a week day and between 8 am and 5 pm.

Your task is to analyze the TEXT below and identify with whom the meeting should be organized and which slots are requested by the user.

Examples:
Text: Arrange a meeting with Florian this week or next week in the afternoon
Participants: Florian
Slots: anyday this week or next week, the afternoon

Text: Organize a meeting with John next Monday at 2 PM
Participants: John
Slots: Monday, 2P M

Your TEXT to analyse:
TEXT: Schedule a meeting with Marc tomorrow morning or Thursday afternoon
Participants:  Marc
Slots: tomorrow morning or Thursday afternoon



Solution:


def findMeeting(text: str) -> dict:
    participants = []
    slots = []

    text = text.lower()
    text = text.replace(" ", "")

    for word in text.split(","):
        if "meeting" in word:
            participants.append(word.split("with ")[-1])
        elif "this" in word

In [31]:
print(response[0]['generated_text'][len(prompt)-14:])

Participants:  Marc
Slots: tomorrow morning or Thursday afternoon


In [36]:
answer = response[0]['generated_text'][len(prompt)-14:]
answer.split("Slots: ")[1]

'tomorrow morning or Thursday afternoon'

# Mistral 7B Instruct

In [ ]:
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
chat = [
    {"role": "system", "content": "You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986."},
    {"role": "user", "content": "Hey, can you tell me any fun things to do in New York?"}
]
sequences = pipe(
    chat,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])